Proses Membuat Kata Kunci 

In [ ]:
import pandas as pd
import mysql.connector 
from mysql.connector import Error

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Fungsi untuk membuat koneksi ke database MySQL
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="jurnal"
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Fungsi untuk membuat tabel jika belum ada
def create_table(cursor):
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS kamus (
        id INT AUTO_INCREMENT PRIMARY KEY,
        kata VARCHAR(255) UNIQUE
    )
    """)

# Fungsi untuk menyimpan kata-kata ke database
def save_words_to_db(cursor, words):
    for word in words:
        try:
            cursor.execute("INSERT INTO kamus (kata) VALUES (%s) ON DUPLICATE KEY UPDATE kata=kata", (word,))
        except Error as e:
            print(f"Error inserting word '{word}': {e}")

# Fungsi untuk memproses teks menjadi kata-kata
def process_text(text):
    words = text.split()
    processed_words = []
    for word in words:
        word = word.strip().lower().replace('.', '').replace(',', '')
        if word:
            processed_words.append(word)
    return processed_words

def main():
    # Path ke file Excel
    file_path = 'C:/xampp/htdocs/autocorrect_search_makalah/data.xlsx'  # Gantilah dengan path ke file Excel Anda

    # Baca file Excel
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' tidak ditemukan.")
        return
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # Asumsikan file Excel memiliki kolom 'Judul' dan 'Abstrak'
    try:
        titles = df['Judul'].tolist()
        abstracts = df['Abstrak'].tolist()
    except KeyError as e:
        print(f"Error: Kolom {e} tidak ditemukan di file Excel.")
        return

    # Membuat koneksi ke database
    db = connect_to_db()
    if db is None:
        return  # Keluar jika tidak bisa terhubung ke database

    cursor = db.cursor()

    # Membuat tabel jika belum ada
    create_table(cursor)

    # Memproses judul dan abstrak dari data
    all_words = []
    for title, abstract in zip(titles, abstracts):
        all_words.extend(process_text(title))
        all_words.extend(process_text(abstract))

    # Menyimpan kata-kata ke database
    save_words_to_db(cursor, all_words)

    # Commit perubahan dan menutup koneksi
    db.commit()
    cursor.close()
    db.close()

if __name__ == "__main__":
    main()
